In [ ]:
import spacy
import pandas as pd 
import numpy as np

In [ ]:
def get_previous_sentence(doc, token_index):
    if doc[token_index].sent.start - 1 < 0:
        return None
    return doc[doc[token_index].sent.start - 1].sent

def get_next_sentence(doc, token_index):
    if doc[token_index].sent.end + 1 >= len(doc):
        return None
    return doc[doc[token_index].sent.end + 1].sent


In [ ]:

nlp = spacy.load('tr_core_news_md')

text =  "Fiber 100mb SuperOnline kullanıcısıyım yaklaşık 2 haftadır @Twitch @Kick_Turkey  gibi canlı yayın platformlarında 360p yayın izlerken donmalar yaşıyoruz Başka hiç bir operatörler bu sorunu yaşamazken ben parasını verip alamadığım hizmeti neden ödeyeyim ? @Turkcell"

doc = nlp(text)

for ent in doc.ents:
    print(ent, ent.label, ent.sent)
    print("ent:", get_previous_sentence(doc, ent.start))
    print("label:", get_next_sentence(doc, ent.start))
    print("---")

In [ ]:
doc.ents

In [ ]:
!pip install https://github.com/turkish-nlp-suite/turkish-spacy-models.git

In [ ]:
!pip install https://huggingface.co/turkish-nlp-suite/tr_core_news_md/resolve/main/tr_core_news_md-any-py3-none-any.whl

# NLTK

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import nltk
from transformers import BertTokenizer, BertForSequenceClassification

# Load the NER model for Turkish
ner_tokenizer = AutoTokenizer.from_pretrained("moarslan/bert-base-turkish-cased-ner")
ner_model = AutoModelForTokenClassification.from_pretrained("moarslan/bert-base-turkish-cased-ner")

# Set up the NER pipeline
ner_pipeline = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer)

# Load a Turkish sentiment analysis model (using BERTurk as an example)
sentiment_tokenizer = BertTokenizer.from_pretrained("")
sentiment_model = BertForSequenceClassification.from_pretrained("")

# Function for sentiment analysis
def sentiment_analysis(sentence):
    inputs = sentiment_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    outputs = sentiment_model(**inputs)
    sentiment = torch.softmax(outputs.logits, dim=-1)
    return sentiment.detach().cpu().numpy()

# Sample text in Turkish
text = "Apple, Türk bir girişim olan U.K.'yi 1 milyar dolara satın almayı düşünüyor. Haberden sonra hisse fiyatları arttı."

# Split text into sentences
nltk.download('punkt')
sentences = nltk.sent_tokenize(text, language="turkish")

# Dictionary to store sentiment for each entity
entity_sentiments = {}

# Process each sentence
for sentence in sentences:
    # Get entities using the NER pipeline
    ner_results = ner_pipeline(sentence)
    
    # Extract entities
    entities = [result['word'] for result in ner_results if result['entity'].startswith("B-")]

    # Perform sentiment analysis for the sentence
    sentiment_score = sentiment_analysis(sentence)

    # Store sentiment score for each entity
    for entity in entities:
        if entity not in entity_sentiments:
            entity_sentiments[entity] = []
        entity_sentiments[entity].append(sentiment_score)

# Optionally, aggregate sentiment scores for each entity
for entity, scores in entity_sentiments.items():
    avg_sentiment = {
        'negative': sum(score[0][0] for score in scores) / len(scores),
        'neutral': sum(score[0][1] for score in scores) / len(scores),
        'positive': sum(score[0][2] for score in scores) / len(scores)
    }
    print(f"Entity: {entity}, Average Sentiment: {avg_sentiment}")
    